# 🌍 Informe de Análisis del Clima en Ciudades del Mundo

## 1. 🔌 Conexión a la API

- Elige una **API de clima**.
- Obtén una **clave API** y pruébala realizando una petición a una sola ciudad.
- Solicita datos como:

  - **Nombre de la ciudad**
  - **Temperatura (°C)**
  - **Humedad (%)**
  - **Velocidad del viento** (m/s o km/h)
  - **Descripción del clima** (por ejemplo, “cielo despejado”)


In [25]:
import requests
import pandas as pd
import country_converter as coco


API_KEY ="a9f821b9935854e146bbe7a6c8ef65b0"

url ="https://api.openweathermap.org/data/2.5/weather"

params ={
    "q": "A Coruña",
    "appid": API_KEY,
    "units": "metric", 
    "lang": "es"        
}

respuesta =requests.get(url, params=params)

data =respuesta.json()
    
weather_info = {
        "Ciudad": data['name'],
        "Temperatura (°C)": data['main']['temp'],
        "Humedad (%)": data['main']['humidity'],
        "Velocidad Viento (m/s)": data['wind']['speed'],
        "Clima Descripcion": data['weather'][0]['description']
}
    
df =pd.DataFrame([weather_info]) 
print(df)


     Ciudad  Temperatura (°C)  Humedad (%)  Velocidad Viento (m/s)  \
0  A Coruña             11.77           82                    4.63   

  Clima Descripcion  
0   nubes dispersas  


## 2. 📝 Recolección de Datos

- Crea una lista de al menos **20 ciudades** de diferentes continentes.  
- Usa un **bucle en Python** para solicitar los datos de cada ciudad y almacenar los resultados.  
- Construye un **DataFrame de pandas** con una fila por cada ciudad.


In [26]:
ciudades = [
    "Madrid", "New York", "Tokyo", "Sydney", "Cairo",
    "Rio de Janeiro", "Moscow", "Cape Town", "Toronto", "Beijing",
    "Mumbai", "Buenos Aires", "Berlin", "Singapore", "Lagos",
    "Mexico City", "Seoul", "Istanbul", "Dubai", "Barcelona"
]
datos_ciudades = []

for ciudad in ciudades:
    params ={
    "q": ciudad,
    "appid": API_KEY,
    "units": "metric", 
    "lang": "es"        
    }

    respuesta =requests.get(url, params=params)

    data =respuesta.json()
        
    ciudades_info = {
        "Ciudad": data['name'],
        "Temperatura (°C)": data['main']['temp'],
        "Humedad (%)": data['main']['humidity'],
        "Velocidad Viento (m/s)": data['wind']['speed'],
        "Clima Descripcion": data['weather'][0]['description']
}
    datos_ciudades.append(ciudades_info)
    


df = pd.DataFrame(datos_ciudades)
print(df)
    

            Ciudad  Temperatura (°C)  Humedad (%)  Velocidad Viento (m/s)  \
0           Madrid             10.91           94                    9.77   
1         New York             10.32           48                    4.63   
2            Tokyo             11.96           69                    3.09   
3           Sydney             17.41           83                    2.57   
4            Cairo             22.42           64                    6.69   
5   Rio de Janeiro             26.98           74                    5.14   
6           Moscow              1.94           55                    6.28   
7        Cape Town             19.97           66                   12.35   
8          Toronto              6.69           72                    4.02   
9          Beijing             -1.06           25                    4.87   
10          Mumbai             28.99           61                    2.06   
11    Buenos Aires             23.51           52                    3.09   

## 3. 🧹 Limpieza de Datos
- Se revisaron los datos en busca de valores faltantes o inconsistentes (por ejemplo, solicitudes fallidas a la API).  
- Se aseguraron formatos correctos para las columnas numéricas.  
- La temperatura se convirtió a **grados Celsius** cuando fue necesario.  
- Los datos finales se guardaron en un archivo **CSV** para su análisis.

In [27]:
datos_ciudades = []
errores = 0
for ciudad in ciudades:
    params = {
        "q": ciudad,
        "appid": API_KEY,
        "units": "metric",
        "lang": "es"
    }

    respuesta = requests.get(url, params=params)
    
    if respuesta.status_code == 200:
        data = respuesta.json()
        ciudades_info = {
            "Pais": data['sys'].get('country'),
            "Ciudad": data.get('name', ciudad),
            "Temperatura (°C)": data['main'].get('temp', None),
            "Humedad (%)": data['main'].get('humidity', None),
            "Velocidad Viento (m/s)": data['wind'].get('speed', None),
            "Clima Descripcion": data['weather'][0].get('description', None)
        }
        datos_ciudades.append(ciudades_info)
    else:
        errores+=1
        print(f"Error en la carga de los datos de {ciudad} (status {respuesta.status_code})")

if errores == 0:
    print(f"No se ha registrado ningún error en las solicitudes de la API")


df = pd.DataFrame(datos_ciudades)
# Convertimos columnas a numéricos
df["Temperatura (°C)"] = pd.to_numeric(df["Temperatura (°C)"], errors='coerce')
df["Humedad (%)"] = pd.to_numeric(df["Humedad (%)"], errors='coerce')
df["Velocidad Viento (m/s)"] = pd.to_numeric(df["Velocidad Viento (m/s)"], errors='coerce')
print(df)

# Eliminar filas con datos críticos faltantes
df.dropna(subset=["Temperatura (°C)", "Humedad (%)"], inplace=True)


# df.to_csv("datos_ciudades.csv", index=False, encoding='utf-8-sig')
    

No se ha registrado ningún error en las solicitudes de la API
   Pais          Ciudad  Temperatura (°C)  Humedad (%)  \
0    ES          Madrid             10.91           94   
1    US        New York             10.32           48   
2    JP           Tokyo             11.96           69   
3    AU          Sydney             17.41           83   
4    EG           Cairo             22.42           64   
5    BR  Rio de Janeiro             26.98           74   
6    RU          Moscow              1.94           55   
7    ZA       Cape Town             19.97           66   
8    CA         Toronto              6.69           72   
9    CN         Beijing             -1.06           25   
10   IN          Mumbai             28.99           61   
11   AR    Buenos Aires             23.51           52   
12   DE          Berlin              2.29           89   
13   SG       Singapore             26.89           85   
14   NG           Lagos             27.80           77   
15   MX   

 #### No he encontrado datos nulos en la API y los datos estas correctamente formateados a numéricos, de todas formas he añadido el código por posibles fallos futuros.

#### Se ha comentado la linea de creación del csv para que no intervenga con posibles aclaciones o respuestas de los próximos apartados

## 4. 📊 Análisis de Datos
Se respondieron preguntas clave:

- 🔥 **Ciudades más calientes y frías:** Se identificaron observando las temperaturas registradas por la API.  
- 💧 **Relación entre temperatura y humedad:** Se analizaron correlaciones entre estas variables.  
- 🌬 **Ciudades más ventosas por continente:** Se compararon las velocidades promedio del viento.  
- ☀️☁️🌧 **Descripción de clima más común:** Se identificó el tipo de clima más frecuente

In [28]:
import pandas as pd

# Cargar datos limpios
df = pd.read_csv("datos_ciudades.csv")

top10_calientes = df.sort_values("Temperatura (°C)", ascending=False).head(10)
print("Top 10 ciudades más calientes:")
print(top10_calientes[["Ciudad", "Temperatura (°C)"]].to_string(index=False))

top10_frias = df.sort_values("Temperatura (°C)", ascending=True).head(10)
print("\nTop 10 ciudades más frías:")
print(top10_frias[["Ciudad", "Temperatura (°C)"]].to_string(index=False))

top10_ventosas = df.sort_values("Velocidad Viento (m/s)", ascending=False).head(10)
print("\nTop 10 ciudades más ventosas:")
print(top10_ventosas[["Ciudad", "Velocidad Viento (m/s)"]].to_string(index=False))

top10_climas = df["Clima Descripcion"].value_counts().head(10)
print("\nTop 10 tipos de clima más frecuentes:")
print(top10_climas.to_string())

cc = coco.CountryConverter()
df["Continente"] = df["Pais"].apply(lambda x: cc.convert(x, to="continent"))

# Top 10 ciudades más ventosas por continente
for continente in df["Continente"].unique():
    print(f"\n🌬 Top 10 ciudades más ventosas en {continente}:")
    top10 = df[df["Continente"] == continente].sort_values("Velocidad Viento (m/s)", ascending=False).head(10)
    print(top10[["Ciudad", "Velocidad Viento (m/s)"]].to_string(index=False))



Top 10 ciudades más calientes:
        Ciudad  Temperatura (°C)
        Mumbai             28.99
         Lagos             27.80
Rio de Janeiro             27.02
     Singapore             26.97
         Dubai             26.96
  Buenos Aires             23.25
         Cairo             22.42
     Cape Town             20.35
   Mexico City             17.75
     Barcelona             17.60

Top 10 ciudades más frías:
  Ciudad  Temperatura (°C)
 Beijing              0.94
  Moscow              1.94
  Berlin              2.17
   Seoul              5.76
 Toronto              5.85
New York             10.09
  Madrid             11.01
   Tokyo             12.18
Istanbul             12.31
  Sydney             17.51

Top 10 ciudades más ventosas:
        Ciudad  Velocidad Viento (m/s)
     Cape Town                   12.35
        Madrid                    9.77
     Barcelona                    7.20
      New York                    6.69
     Singapore                    6.69
        Moscow  

KeyError: 'Pais'

## 5. 📈 Visualización de Datos
Se generaron gráficos para apoyar el análisis:

- **Gráfico de barras:** temperatura promedio por continente 🌡📊  
- **Diagrama de dispersión:** temperatura vs humedad 💧🌡

## 6. 📝 Resumen
- **Proceso:** Se recopilaron datos mediante la API, se limpiaron y normalizaron, y se realizó un análisis exploratorio.  
- **Patrones interesantes:** Se observaron diferencias de temperatura y viento entre continentes y se identificó la descripción de clima más frecuente.  
- **Dificultades o limitaciones:** Algunas solicitudes a la API pudieron fallar o devolver datos incompletos. La muestra de 20 ciudades no es representativa de todo el mundo, por lo que los resultados son indicativos y no definitivos